### Context

#### Experiment Tools

- wandb

In [1]:
# !pip install wandb

- json 파일로 관리하면 좋음

In [2]:
sweep_config = {
  "name" : "mdc_sweep",
  "method" : "bayes",
  "parameters" : {
    "max_depth" : {
      "distribution": "int_uniform",
      "min":2,
      "max":15
    },
    "subsample" :{
      "distribution": "uniform",
      "min": 0.5,
      "max": 1.0
    },
    "colsample_bytree":{
      "distribution": "uniform",
      "min": 0.5,
      "max": 1.0
    }
  },
  "metric":{
      "name": "cv_loss",
      "goal": "minimize"
  }
}


In [3]:
import os
from os.path import join

import multiprocessing
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

import wandb

# 웹 서버 연결
wandb.init()

# optuna 세팅
sweep_id = wandb.sweep(sweep_config, 
                       project="medici wandb test")

n_cpus = multiprocessing.cpu_count()

wandb: Currently logged in as: kendrickaskedforgeotjeori (use `wandb login --relogin` to force relogin)


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Create sweep with ID: y129mdb1
Sweep URL: https://wandb.ai/kendrickaskedforgeotjeori/medici%20wandb%20test/sweeps/y129mdb1


In [4]:
BASE_DIR = './data' 

train_path = os.path.join(BASE_DIR, 'train.csv')
test_path = os.path.join(BASE_DIR, 'test.csv')
submission_path = os.path.join(BASE_DIR, 'sample_submission.csv')

data = pd.read_csv(train_path)
test = pd.read_csv(test_path)

label = data['credit']

In [5]:
# 불필요한 컬럼 제거
data.drop(columns=['index', 'credit'], inplace=True)
test.drop(columns=['index'],         inplace=True)

In [6]:
cat_columns = [c for c, t in zip(data.dtypes.index, data.dtypes) if t == 'O'] 
num_columns = [c for c    in data.columns if c not in cat_columns]

print('Categorical Columns: \n{}\n'.format(cat_columns))
print('Numeric Columns: \n{}'.format(num_columns))

Categorical Columns: 
['gender', 'car', 'reality', 'income_type', 'edu_type', 'family_type', 'house_type', 'occyp_type']

Numeric Columns: 
['child_num', 'income_total', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'FLAG_MOBIL', 'work_phone', 'phone', 'email', 'family_size', 'begin_month']


#### 라벨 데이터 인코딩

In [7]:
label = label.astype(int)

#### 전처리 프로세스 함수로 작성

In [8]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

def preprocess(x_train, x_valid, x_test):
    tmp_x_train = x_train.copy()
    tmp_x_valid = x_valid.copy()
    tmp_x_test  = x_test.copy()
    
    tmp_x_train.reset_index(drop=True, inplace=True)
    tmp_x_valid.reset_index(drop=True, inplace=True)
    
    # 결측치 처리
    imputer = SimpleImputer(strategy='most_frequent')
    tmp_x_train[cat_columns] = imputer.fit_transform(tmp_x_train[cat_columns])
    tmp_x_valid[cat_columns] = imputer.transform(tmp_x_valid[cat_columns])
    tmp_x_test[cat_columns]  = imputer.transform(tmp_x_test[cat_columns])
    
    # 스케일링
    scaler = StandardScaler()
    tmp_x_train[num_columns] = scaler.fit_transform(tmp_x_train[num_columns])
    tmp_x_valid[num_columns] = scaler.transform(tmp_x_valid[num_columns])
    tmp_x_test[num_columns]  = scaler.transform(tmp_x_test[num_columns])

    # 인코딩
    ohe = OneHotEncoder(sparse=False)
    ohe.fit(tmp_x_train[cat_columns])
    
    tmp_x_train_cat = pd.DataFrame(ohe.transform(tmp_x_train[cat_columns]))
    tmp_x_valid_cat = pd.DataFrame(ohe.transform(tmp_x_valid[cat_columns]))
    tmp_x_test_cat  = pd.DataFrame(ohe.transform(tmp_x_test[cat_columns]))
    
    tmp_x_train.drop(columns=cat_columns, inplace=True)
    tmp_x_valid.drop(columns=cat_columns, inplace=True)
    tmp_x_test.drop(columns=cat_columns, inplace=True)
    
    tmp_x_train = pd.concat([tmp_x_train, tmp_x_train_cat], axis=1)
    tmp_x_valid = pd.concat([tmp_x_valid, tmp_x_valid_cat], axis=1)
    tmp_x_test  = pd.concat([tmp_x_test, tmp_x_test_cat], axis=1)
    
    return tmp_x_train, tmp_x_valid, tmp_x_test

### Ensemble

In [9]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
from xgboost import XGBClassifier

def train():
    with wandb.init() as run:
        params = wandb.config
        
        val_scores = []
        n_splits = 5

        skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

        for i, (trn_idx, val_idx) in enumerate(skf.split(data, label)):
            x_train, y_train = data.iloc[trn_idx, :], label.iloc[trn_idx,]
            x_valid, y_valid = data.iloc[val_idx, :], label.iloc[val_idx,]

            # 전처리
            x_train, x_valid, x_test = preprocess(x_train, x_valid, test)

            # 모델 정의
            model = XGBClassifier(n_estimators=1000,
                                  max_depth=params['max_depth'],
                                  subsample=params['subsample'],
                                  colsample_bytree=params['colsample_bytree'],
#                                   tree_method='gpu_hist',
                                  n_jobs=n_cpus-1)

            # 모델 학습
            model.fit(x_train, y_train, 
                      eval_metric='mlogloss', 
                      eval_set=[[x_train, y_train], [x_valid, y_valid]],
                      early_stopping_rounds=100,
                      verbose=100)

            # 훈련, 검증 데이터 log_loss 확인
            trn_logloss = log_loss(y_train, model.predict_proba(x_train))
            val_logloss = log_loss(y_valid, model.predict_proba(x_valid))
            print('{} Fold, train logloss : {:.4f}4, validation logloss : {:.4f}'.format(i, trn_logloss, val_logloss))

            val_scores.append(val_logloss)
            
        metrics = {"cv_loss": np.mean(val_scores)}
        wandb.log(metrics)
count = 5
wandb.agent(sweep_id, function=train, count=count)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: Agent Starting Run: wgal4hhf with config:
wandb: 	colsample_bytree: 0.5756749462422828
wandb: 	max_depth: 3
wandb: 	subsample: 0.5852859397198615


[0]	validation_0-mlogloss:0.97893	validation_1-mlogloss:0.97752
[100]	validation_0-mlogloss:0.73752	validation_1-mlogloss:0.77913
[200]	validation_0-mlogloss:0.69462	validation_1-mlogloss:0.76673
[300]	validation_0-mlogloss:0.66307	validation_1-mlogloss:0.76092
[400]	validation_0-mlogloss:0.63506	validation_1-mlogloss:0.75506
[500]	validation_0-mlogloss:0.61140	validation_1-mlogloss:0.74666
[600]	validation_0-mlogloss:0.59180	validation_1-mlogloss:0.74569
[700]	validation_0-mlogloss:0.57372	validation_1-mlogloss:0.74429
[789]	validation_0-mlogloss:0.55990	validation_1-mlogloss:0.74470
0 Fold, train logloss : 0.57554, validation logloss : 0.7436
[0]	validation_0-mlogloss:0.97831	validation_1-mlogloss:0.98028
[100]	validation_0-mlogloss:0.73282	validation_1-mlogloss:0.79068
[200]	validation_0-mlogloss:0.69027	validation_1-mlogloss:0.77834
[300]	validation_0-mlogloss:0.65877	validation_1-mlogloss:0.77136
[400]	validation_0-mlogloss:0.63210	validation_1-mlogloss:0.76626
[500]	validation_0-

cv_loss,0.75054
_runtime,202
_timestamp,1631542518
_step,0


cv_loss,▁
_runtime,▁
_timestamp,▁
_step,▁


wandb: Agent Starting Run: o2r5krvz with config:
wandb: 	colsample_bytree: 0.7563521143590718
wandb: 	max_depth: 10
wandb: 	subsample: 0.7533199499098471


...

cv_loss,▁
_runtime,▁
_timestamp,▁
_step,▁


wandb: Agent Starting Run: 7iz934q9 with config:
wandb: 	colsample_bytree: 0.909678292874907
wandb: 	max_depth: 15
wandb: 	subsample: 0.6387906182499608


[0]	validation_0-mlogloss:0.94245	validation_1-mlogloss:0.96872
[100]	validation_0-mlogloss:0.09132	validation_1-mlogloss:0.92934
[118]	validation_0-mlogloss:0.07986	validation_1-mlogloss:0.96632
0 Fold, train logloss : 0.40734, validation logloss : 0.7252
[0]	validation_0-mlogloss:0.94029	validation_1-mlogloss:0.97132
[100]	validation_0-mlogloss:0.09449	validation_1-mlogloss:0.96153
[115]	validation_0-mlogloss:0.08394	validation_1-mlogloss:0.99616
1 Fold, train logloss : 0.43644, validation logloss : 0.7394
[0]	validation_0-mlogloss:0.96266	validation_1-mlogloss:0.99130
[100]	validation_0-mlogloss:0.09438	validation_1-mlogloss:0.95826
[116]	validation_0-mlogloss:0.08304	validation_1-mlogloss:0.99514
2 Fold, train logloss : 0.43464, validation logloss : 0.7422
[0]	validation_0-mlogloss:0.96647	validation_1-mlogloss:0.99432
[100]	validation_0-mlogloss:0.09265	validation_1-mlogloss:0.95379
[117]	validation_0-mlogloss:0.08073	validation_1-mlogloss:0.99649
3 Fold, train logloss : 0.40034, 

cv_loss,0.73555
_runtime,140
_timestamp,1631543063
_step,0


cv_loss,▁
_runtime,▁
_timestamp,▁
_step,▁


In [ ]:
submit.to_csv('oof_first_submit.csv', index=False)